In [1]:
import sys 
sys.executable

'C:\\Users\\Changho_Notebook\\anaconda3\\python.exe'

In [2]:
import pandas as pd 
import numpy as np 
%matplotlib inline
import matplotlib.pyplot as plt
import re 
import urllib.request
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [3]:
data = pd.read_csv('./data/test_data0413.csv', encoding = 'euc-kr')

In [4]:
data

,Text,Feeling,Talking,book_id,sentence_id
0,"‘인도’가 적에게 포위를 당했는데, 적들이 은화 6백 개를 내놓을 때까지 물러나지 ...",2.0,0,5001,1.0
1,"그래서 주민들이 북을 치며 선포하길,",2.0,0,5001,2.0
2,*누구든 그 돈을 마련하는 사람을 시장으로 삼겠다*,2.0,1,5001,3.0
3,라고 했답니다,0.0,0,5001,4.0
4,"때마침 가난한 어부가 자신의 아들과 호수에서 물고기를 잡아 생활하다, 그 적들이 들...",5.0,0,5001,5.0
...,...,...,...,...,...
13053,그런데 그 호랑이는 바로 옛날에 나무꾼이 도와주었던 호랑이였습니다,4.0,0,7009,30.0
13054,*저는 나이가 들어 어차피 죽을 목숨이니 저를 임금에게 제물로 바치십시오*,4.0,1,7009,31.0
13055,그러곤 호랑이는 숨을 거두었습니다,4.0,0,7009,32.0
13056,나무꾼은 숨을 거둔 호랑이를 차마 제물로 바치지 못하고 양지바른 땅에 묻어주었습니다,4.0,0,7009,33.0


In [5]:
data_df = data['Text']

In [6]:
data_df

0        ‘인도’가 적에게 포위를 당했는데, 적들이 은화 6백 개를 내놓을 때까지 물러나지 ...
1                                    그래서 주민들이 북을 치며 선포하길, 
2                             *누구든 그 돈을 마련하는 사람을 시장으로 삼겠다*
3                                                  라고 했답니다
4        때마침 가난한 어부가 자신의 아들과 호수에서 물고기를 잡아 생활하다, 그 적들이 들...
                               ...                        
13053                 그런데 그 호랑이는 바로 옛날에 나무꾼이 도와주었던 호랑이였습니다
13054            *저는 나이가 들어 어차피 죽을 목숨이니 저를 임금에게 제물로 바치십시오*
13055                                   그러곤 호랑이는 숨을 거두었습니다
13056       나무꾼은 숨을 거둔 호랑이를 차마 제물로 바치지 못하고 양지바른 땅에 묻어주었습니다
13057    목에 걸린 뼈를 나무꾼이 빼준 이후로 죽을 때까지 나무꾼에게 은혜를 갚는 호랑이의 ...
Name: Text, Length: 13058, dtype: object

In [124]:
import requests, json
 
# URL 지정
url = 'http://svc.saltlux.ai:31781'
 
# Header 정보 지정
headers = {'Content-Type': 'application/json; charset=utf-8'}
 
# Request Parameter 정보 지정
params = {
    "key": "389dddd9-70ee-46a9-9140-9713b9932b02",
    "serviceId": "11987300804",
    "argument": {
        "type": "1",
        "query": " *누구든 시장님이라고 불러야 한다"
    }
}
 
response = requests.post(url, headers = headers, data = json.dumps(params))
 
type(response.content) # bytes 이여서 decode 작업을 해야함
    
y = response.content
z = y.decode('utf-8')
dict = json.loads(z)
dict

{'query': ' *누구든 시장님이라고 불러야 한다',
 'type': '감정분석',
 'Result': [[0.6897064447402954, '분노']]}

In [125]:
test_df = pd.DataFrame(dict)
test_df

,query,type,Result
0,*누구든 시장님이라고 불러야 한다,감정분석,"[0.6897064447402954, 분노]"


In [126]:
result_df = test_df['Result']
result_df.str.split(',')
a = result_df[0]
a_df = a[0]
b_df = a[1]

In [127]:
test_df['Result'] = a_df
test_df

,query,type,Result
0,*누구든 시장님이라고 불러야 한다,감정분석,0.689706


In [128]:
test_df['Feel'] = b_df
test_df

,query,type,Result,Feel
0,*누구든 시장님이라고 불러야 한다,감정분석,0.689706,분노


In [231]:
url = 'http://svc.saltlux.ai:31781'
headers = {'Content-Type': 'application/json; charset=utf-8'}

test_df = []
test_df_full = []

for idx in data_df:
    params = {
    "key": "389dddd9-70ee-46a9-9140-9713b9932b02",
    "serviceId": "11987300804",
    "argument": {
        "type": "1",
        "query": idx
        }
    }
    response = requests.post(url, headers = headers, data = json.dumps(params))
    y = response.content
    z = y.decode('utf-8')
    dict = json.loads(z)
    # print(dict)
    test_df.append(dict['Result'][0][1])
    test_df_full.append(dict)
print(test_df)

['분노', '공포', '기대', '신뢰', '혐오', '혐오', '혐오', '기쁨', '분노', '혐오', '신뢰', '신뢰', '신뢰', '혐오', '혐오', '분노', '신뢰', '기대', '혐오', '신뢰', '혐오', '기대', '신뢰', '기대', '신뢰', '혐오', '신뢰', '기대', '혐오', '신뢰', '혐오', '신뢰', '슬픔', '혐오', '혐오', '혐오', '신뢰', '분노', '분노', '기쁨', '분노', '혐오', '신뢰', '분노', '기대', '혐오', '분노', '혐오', '분노', '기대', '혐오', '혐오', '신뢰', '공포', '신뢰', '신뢰', '신뢰', '혐오', '혐오', '기대', '혐오', '공포', '혐오', '신뢰', '신뢰', '신뢰', '신뢰', '혐오', '신뢰', '혐오', '공포', '혐오', '신뢰', '혐오', '분노', '신뢰', '신뢰', '공포', '분노', '신뢰', '혐오', '혐오', '혐오', '혐오', '신뢰', '공포', '신뢰', '신뢰', '혐오', '분노', '혐오', '혐오', '혐오', '혐오', '혐오', '신뢰', '혐오', '신뢰', '혐오', '신뢰', '혐오', '기대', '신뢰', '혐오', '신뢰', '신뢰', '혐오', '혐오', '신뢰', '혐오', '신뢰', '슬픔', '혐오', '슬픔', '슬픔', '신뢰', '혐오', '신뢰', '신뢰', '신뢰', '신뢰', '기대', '신뢰', '신뢰', '신뢰', '신뢰', '신뢰', '신뢰', '공포', '혐오', '신뢰', '혐오', '혐오', '혐오', '신뢰', '혐오', '신뢰', '신뢰', '신뢰', '분노', '분노', '혐오', '혐오', '신뢰', '신뢰', '신뢰', '신뢰', '혐오', '신뢰', '혐오', '분노', '신뢰', '신뢰', '신뢰', '신뢰', '기대', '신뢰', '신뢰', '혐오', '혐오', '혐오', '혐오', '신뢰', '분노', '신뢰', '분노', '분노

In [232]:
print(test_df_full)

[{'query': '‘인도’가 적에게 포위를 당했는데, 적들이 은화 6백 개를 내놓을 때까지 물러나지 않겠다고 말했어요', 'type': '감정분석', 'Result': [[0.4068281054496765, '분노']]}, {'query': '그래서 주민들이 북을 치며 선포하길, ', 'type': '감정분석', 'Result': [[0.3689020872116089, '공포']]}, {'query': '*누구든 그 돈을 마련하는 사람을 시장으로 삼겠다*', 'type': '감정분석', 'Result': [[0.8977451920509338, '기대']]}, {'query': '라고 했답니다', 'type': '감정분석', 'Result': [[0.6649170517921448, '신뢰'], [0.6280138492584229, '기쁨']]}, {'query': '때마침 가난한 어부가 자신의 아들과 호수에서 물고기를 잡아 생활하다, 그 적들이 들이 닫쳐 아들을 포로로 잡아가며 대신 은화 6백 개를 아빠에게 주고 갔어요', 'type': '감정분석', 'Result': [[0.5542007088661194, '혐오']]}, {'query': '그래서 아빠가 가서 도시의 지도자들에게 그 돈을 건넸지요', 'type': '감정분석', 'Result': [[0.3958638310432434, '혐오']]}, {'query': '결국 적들은 물러가고, 어부는 시장이 되었답니다', 'type': '감정분석', 'Result': [[0.5343120694160461, '혐오']]}, {'query': '그리하여 ', 'type': '감정분석', 'Result': [[0.20184090733528137, '기쁨']]}, {'query': '*누구든 시장님이라고 불러야 한다', 'type': '감정분석', 'Result': [[0.6897064447402954, '분노']]}, {'query': ' 그러지 않으면 교수형에 처하리라*', 'type': '감정분석', 'Res

In [251]:
test_df = pd.DataFrame (test_df)
test_df

,0
0,기대
1,혐오
2,신뢰
3,신뢰
4,혐오
...,...
4053,신뢰
4054,분노
4055,신뢰
4056,혐오


In [237]:
test_df.to_csv('./test_df_9000.csv', index=False, sep = ',', encoding =  'euc-kr')

In [238]:
test_df_full = pd.DataFrame (test_df_full)
test_df_full

,query,type,Result
0,"‘인도’가 적에게 포위를 당했는데, 적들이 은화 6백 개를 내놓을 때까지 물러나지 ...",감정분석,"[[0.4068281054496765, 분노]]"
1,"그래서 주민들이 북을 치며 선포하길,",감정분석,"[[0.3689020872116089, 공포]]"
2,*누구든 그 돈을 마련하는 사람을 시장으로 삼겠다*,감정분석,"[[0.8977451920509338, 기대]]"
3,라고 했답니다,감정분석,"[[0.6649170517921448, 신뢰], [0.6280138492584229..."
4,"때마침 가난한 어부가 자신의 아들과 호수에서 물고기를 잡아 생활하다, 그 적들이 들...",감정분석,"[[0.5542007088661194, 혐오]]"
...,...,...,...
8995,"이제 제 마음을 휘저은 광경이 그 애란 걸 당신도 아셨으니, 말씀해주세요, 당신은...",감정분석,"[[0.8249478936195374, 분노]]"
8996,백작은 숲에서 어느 노파를 만나 그 책을 건네받게 되었다고 말했어요,감정분석,"[[0.3381275534629822, 혐오]]"
8997,"*그 노파는 무척 기이한 여자로, 아무래도 마녀 같았습니다",감정분석,"[[0.4905655086040497, 신뢰]]"
8998,하지만 공주님에 관한 소식은 어디서도 전해 듣거나 눈으로 확인한 적은 없습니다*,감정분석,"[[0.8406078815460205, 기대]]"


In [239]:
test_df_full.to_csv('./test_df_full_9000.csv', index=False, sep = ',', encoding =  'euc-kr')

In [246]:
data_df_2nd = data['Text'][9000:]

In [247]:
data_df_2nd

9000           진주가 있었던 곳이 거기니 딸애의 소식도 전해들을 수 있을 거라 믿었던 거지요
9001                    한편 그 시각, 노파는 홀로 물레에 앉아 실을 만들고 있었어요
9002                                            땅거미가 진 후였죠
9003                           난로에서 장작들이 타며 부족한 빛을 내주고 있었죠
9004     곧 밖에서 소리가 들리더니, 거위들이 풀밭에서 집으로 돌아왔는지 한동안 꽥꽥거리는 ...
                               ...                        
13053                 그런데 그 호랑이는 바로 옛날에 나무꾼이 도와주었던 호랑이였습니다
13054            *저는 나이가 들어 어차피 죽을 목숨이니 저를 임금에게 제물로 바치십시오*
13055                                   그러곤 호랑이는 숨을 거두었습니다
13056       나무꾼은 숨을 거둔 호랑이를 차마 제물로 바치지 못하고 양지바른 땅에 묻어주었습니다
13057    목에 걸린 뼈를 나무꾼이 빼준 이후로 죽을 때까지 나무꾼에게 은혜를 갚는 호랑이의 ...
Name: Text, Length: 4058, dtype: object

In [248]:
url = 'http://svc.saltlux.ai:31781'
headers = {'Content-Type': 'application/json; charset=utf-8'}

test_df_2nd = []
test_df_full_2nd = []

for idx in data_df_2nd:
    params = {
    "key": "3e1efc54-98d9-4de4-a111-97728a14f0b2",
    "serviceId": "11987300804",
    "argument": {
        "type": "1",
        "query": idx
        }
    }
    response = requests.post(url, headers = headers, data = json.dumps(params))
    y = response.content
    z = y.decode('utf-8')
    dict = json.loads(z)
    # print(dict)
    test_df_2nd.append(dict['Result'][0][1])
    test_df_full_2nd.append(dict)
print(test_df_2nd)

['기대', '혐오', '신뢰', '신뢰', '혐오', '신뢰', '신뢰', '혐오', '혐오', '혐오', '혐오', '기쁨', '신뢰', '혐오', '신뢰', '공포', '신뢰', '신뢰', '혐오', '신뢰', '신뢰', '혐오', '신뢰', '신뢰', '신뢰', '신뢰', '신뢰', '신뢰', '신뢰', '신뢰', '혐오', '신뢰', '혐오', '혐오', '혐오', '혐오', '분노', '혐오', '신뢰', '신뢰', '혐오', '신뢰', '혐오', '분노', '분노', '혐오', '신뢰', '혐오', '신뢰', '신뢰', '혐오', '신뢰', '신뢰', '분노', '신뢰', '분노', '신뢰', '혐오', '신뢰', '혐오', '분노', '혐오', '분노', '혐오', '신뢰', '기대', '기대', '공포', '혐오', '혐오', '신뢰', '분노', '혐오', '공포', '혐오', '신뢰', '신뢰', '기대', '신뢰', '신뢰', '혐오', '혐오', '혐오', '신뢰', '공포', '혐오', '공포', '신뢰', '신뢰', '신뢰', '혐오', '혐오', '혐오', '신뢰', '신뢰', '신뢰', '신뢰', '혐오', '신뢰', '신뢰', '혐오', '분노', '신뢰', '신뢰', '신뢰', '신뢰', '신뢰', '슬픔', '혐오', '기대', '신뢰', '신뢰', '신뢰', '신뢰', '신뢰', '신뢰', '혐오', '기대', '혐오', '혐오', '슬픔', '신뢰', '신뢰', '신뢰', '혐오', '신뢰', '신뢰', '신뢰', '공포', '신뢰', '신뢰', '혐오', '신뢰', '기대', '기대', '신뢰', '신뢰', '신뢰', '신뢰', '혐오', '혐오', '신뢰', '혐오', '신뢰', '신뢰', '신뢰', '신뢰', '신뢰', '분노', '기대', '신뢰', '분노', '신뢰', '공포', '신뢰', '분노', '신뢰', '분노', '신뢰', '분노', '신뢰', '분노', '신뢰', '분노', '신뢰', '신뢰', '기대

In [250]:
test_df = pd.DataFrame (test_df_2nd)
test_df

,0
0,기대
1,혐오
2,신뢰
3,신뢰
4,혐오
...,...
4053,신뢰
4054,분노
4055,신뢰
4056,혐오


In [254]:
test_df.to_csv('./test_df_9000_2nd.csv', index=False, sep = ',', encoding =  'euc-kr')

In [255]:
test_df_full_2nd = pd.DataFrame (test_df_full_2nd)
test_df_full_2nd

,query,type,Result
0,진주가 있었던 곳이 거기니 딸애의 소식도 전해들을 수 있을 거라 믿었던 거지요,감정분석,"[[0.9743438363075256, 기대]]"
1,"한편 그 시각, 노파는 홀로 물레에 앉아 실을 만들고 있었어요",감정분석,"[[0.8546315431594849, 혐오]]"
2,땅거미가 진 후였죠,감정분석,"[[0.2863691747188568, 신뢰]]"
3,난로에서 장작들이 타며 부족한 빛을 내주고 있었죠,감정분석,"[[0.5449281930923462, 신뢰]]"
4,"곧 밖에서 소리가 들리더니, 거위들이 풀밭에서 집으로 돌아왔는지 한동안 꽥꽥거리는 ...",감정분석,"[[0.9383059740066528, 혐오]]"
...,...,...,...
4053,그런데 그 호랑이는 바로 옛날에 나무꾼이 도와주었던 호랑이였습니다,감정분석,"[[0.8775262832641602, 신뢰]]"
4054,*저는 나이가 들어 어차피 죽을 목숨이니 저를 임금에게 제물로 바치십시오*,감정분석,"[[0.6633353233337402, 분노]]"
4055,그러곤 호랑이는 숨을 거두었습니다,감정분석,"[[0.7068330645561218, 신뢰]]"
4056,나무꾼은 숨을 거둔 호랑이를 차마 제물로 바치지 못하고 양지바른 땅에 묻어주었습니다,감정분석,"[[0.7573866844177246, 혐오]]"


In [256]:
test_df_full_2nd.to_csv('./test_df_full_9000_2nd.csv', index=False, sep = ',', encoding =  'euc-kr')

In [257]:
data = pd.read_csv('./data/청와대_전래동화.csv', encoding = 'euc-kr')

In [258]:
data

,Text,Feeling,Talking,book_id,sentence_id
0,별난 재주 삼형제옛날 시골마을에 별난 재주 세 형제가 살았습니다,1.0,NaN,NaN,NaN
1,눈이 밝은 맏이는 별별 것을 다 봅니다,1.0,NaN,NaN,NaN
2,둘째는 천하장사 바윗돌도 번쩍번쩍,1.0,NaN,NaN,NaN
3,개구쟁이 막내는 희한한 재주인데 매 맞는 재주입니다,1.0,NaN,NaN,NaN
4,회초리로 때리면 간지럽다고 깔깔대고 절굿공이 내리치면 시원하다 깔깔댑니다,1.0,NaN,NaN,NaN
...,...,...,...,...,...
3638,그리고 다음 박에선 온갖 더러운 오물들이 넘쳐나서 놀부의 집은 완전 아수라장이 되...,NaN,NaN,NaN,NaN
3639,하루아침만에 놀부는 가난뱅이가 되어버렸습니다,NaN,NaN,NaN,NaN
3640,놀부는 끝없는 욕심을 부리다가 이렇게 벌을 받게 되었죠,NaN,NaN,NaN,NaN
3641,"비록 형인 놀부에겐 천대받았던 흥부였지만, 이 소식을 들은 심성이 착한 흥부는 내...",NaN,NaN,NaN,NaN


In [280]:
data_df_3nd = data['Text'][2000:]
data_df_3nd

2000                                                    *
2001     마을 대표는 산을 내려와 주민들과 함께 부림말 우물에 귀한 소머리를 놓고 제사를 ...
2002         가까이에 있는 박우물과 신작로 우물에는 소의 갈비를 사용해 고사를 지내게 됩니다
2003     그랬더니 아니나 다를까, 신기하게도 물이 솟구치며 올라와 우물을 가득 채워졌던 것...
2004                       올해 우리나라도 가뭄이 들어 농촌에서는 고생이 많았지요
                              ...                        
3638     그리고 다음 박에선 온갖 더러운 오물들이 넘쳐나서 놀부의 집은 완전 아수라장이 되...
3639                             하루아침만에 놀부는 가난뱅이가 되어버렸습니다
3640                       놀부는 끝없는 욕심을 부리다가 이렇게 벌을 받게 되었죠
3641     비록 형인 놀부에겐 천대받았던 흥부였지만, 이 소식을 들은 심성이 착한 흥부는 내...
3642     놀부처럼 너무 욕심을 부려서 재물을 탐내기 보다는 흥부처럼 용서하고 베푸는 마음을...
Name: Text, Length: 1643, dtype: object

In [281]:
url = 'http://svc.saltlux.ai:31781'
headers = {'Content-Type': 'application/json; charset=utf-8'}

test_df_3nd = []
test_df_full_3nd = []

for idx in data_df_3nd:
    params = {
    "key": "3e1efc54-98d9-4de4-a111-97728a14f0b2",
    "serviceId": "11987300804",
    "argument": {
        "type": "1",
        "query": idx
        }
    }
    response = requests.post(url, headers = headers, data = json.dumps(params))
    y = response.content
    z = y.decode('utf-8')
    dict = json.loads(z)
    # print(dict)
    test_df_3nd.append(dict['Result'][0][1])
    test_df_full_3nd.append(dict)
print(test_df_3nd)

['기쁨', '신뢰', '기대', '신뢰', '공포', '기대', '혐오', '혐오', '혐오', '분노', '공포', '신뢰', '신뢰', '신뢰', '공포', '분노', '신뢰', '공포', '분노', '신뢰', '신뢰', '신뢰', '신뢰', '신뢰', '신뢰', '혐오', '혐오', '신뢰', '혐오', '분노', '신뢰', '분노', '신뢰', '분노', '신뢰', '혐오', '분노', '신뢰', '공포', '혐오', '분노', '신뢰', '분노', '분노', '기대', '혐오', '혐오', '기대', '혐오', '혐오', '신뢰', '혐오', '신뢰', '혐오', '신뢰', '분노', '혐오', '신뢰', '혐오', '혐오', '혐오', '혐오', '신뢰', '신뢰', '신뢰', '신뢰', '신뢰', '혐오', '혐오', '신뢰', '신뢰', '신뢰', '혐오', '혐오', '혐오', '혐오', '신뢰', '기대', '혐오', '신뢰', '신뢰', '신뢰', '신뢰', '신뢰', '신뢰', '신뢰', '신뢰', '신뢰', '신뢰', '신뢰', '기대', '신뢰', '신뢰', '기대', '분노', '신뢰', '혐오', '신뢰', '신뢰', '신뢰', '혐오', '혐오', '혐오', '혐오', '혐오', '혐오', '분노', '신뢰', '신뢰', '혐오', '신뢰', '신뢰', '신뢰', '혐오', '혐오', '혐오', '혐오', '신뢰', '혐오', '신뢰', '기대', '신뢰', '혐오', '분노', '분노', '신뢰', '신뢰', '혐오', '혐오', '혐오', '혐오', '혐오', '혐오', '혐오', '혐오', '분노', '신뢰', '신뢰', '신뢰', '기대', '혐오', '혐오', '혐오', '분노', '신뢰', '혐오', '혐오', '혐오', '혐오', '혐오', '혐오', '신뢰', '신뢰', '신뢰', '혐오', '신뢰', '혐오', '신뢰', '신뢰', '혐오', '분노', '기대', '신뢰', '혐오', '혐오', '혐오', '슬픔

In [ ]:
test_df = pd.DataFrame (test_df_3nd)
test_df

,0
0,기쁨
1,신뢰
2,기대
3,신뢰
4,공포
...,...
1638,혐오
1639,혐오
1640,혐오
1641,신뢰


In [ ]:
test_df.to_csv('./test_df_5nd.csv', index=False, sep = ',', encoding =  'euc-kr')